# Resumir textos grandes con Langchain

## Instalación de paquetes
Si estás corriendo este notebook en Google Colab, corre la siguiente celda para instalar los paquetes necesarios.

In [1]:
# %pip install langchain langchain_community langchain_openai

In [2]:
# Corre esta celda solo si tienes un archivo .env configurado
from dotenv import load_dotenv
load_dotenv()

True

### Definición de Contexto

En el ámbito de los LLMs, la "ventana de contexto" representa el tamaño máximo del prompt proporcionado a un LLM, que incluye instrucciones y contexto. Diferentes LLMs tienen límites de tokens distintos para la ventana de contexto: GPT-3.5 solía aceptar hasta 16K tokens, GPT-4o y LLama 3 hasta 128K, y Gemini-1.5 hasta 1M.

## Resumir un documento grande

Vamos a resumir el libro "Cien años de soledad" de Gabriel García Márquez. Que puedes descargar en el siguiente enlace (en formato txt y cortado a 18.000 palabras): [Cien años de soledad](https://gist.githubusercontent.com/ismaproco/6781d297ee65c6a707cd3c901e87ec56/raw/20d3520cd7c53d99215845375b1dca16ac827bd7/gabriel_garcia_marquez_cien_annos_soledad.txt)

Lo que vamos a realizar de manera simplificada es lo siguiente:

1. Dividir el texto grandes en documentos pequeños (3000 tokens)
2. Generar resúmenes en paralelo de cada documento pequeño (operación Map)
3. Reducir todos los resúmenes a un solo resumen (operación Reduce)

Esta técnica es conocida como ***MapReduce***.

In [4]:
with open("../../datasets/cien_annos_soledad_reducido.txt", 'r', encoding='utf-8') as f:
    book = f.read()

In [5]:
# Importa las librerías necesarias

import os
from langchain_openai import ChatOpenAI
from langchain_text_splitters import TokenTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel
import getpass

In [6]:
llm = ChatOpenAI(
    model=os.getenv("MODEL"),
    openai_api_key=os.getenv("LIA_API_KEY"),
    openai_api_base=os.getenv("LIA_API_BASE"),
    max_tokens=2000,
    temperature=0.6,
)

In [7]:
text_chunks_chain = (
    RunnableLambda(lambda x: 
        [
            {
                'chunk': text_chunk, 
            }
            for text_chunk in 
               TokenTextSplitter(chunk_size=3000, chunk_overlap=100).split_text(x)
        ]
    )
)

In [8]:
# Es lo mismo que si escribiéramos:

def split_into_chunks(text):
    
    # Crear un divisor de texto que usa los parámetros chunk_size y chunk_overlap
    splitter = TokenTextSplitter(chunk_size=3000, chunk_overlap=100)
    
    # Dividir el texto en fragmentos
    chunks = splitter.split_text(text)
    
    # Devolver cada fragmento como un diccionario
    return [{'chunk': text_chunk} for text_chunk in chunks]

# Convertir la función en un componente 'Runnable' usando RunnableLambda
text_chunks_chain = RunnableLambda(split_into_chunks)

In [9]:
# El siguiente paso es configurar la cadena de mapeo, la cual ejecutará una solicitud de resumen para cada fragmento de documento:


plantilla_resumir_fragmento = """
Escribe un resumen conciso del siguiente texto, e incluye los detalles principales.
Texto: {chunk}
"""
 
resumir_fragmento_prompt = PromptTemplate.from_template(plantilla_resumir_fragmento)
cadena_resumir_fragmento = resumir_fragmento_prompt | llm
 
cadena_mapa_resumir = (
    RunnableParallel (
        {
            'resumen': cadena_resumir_fragmento | StrOutputParser()     
        }
    )
)


Ahora vamos a configurar la cadena de reducción (reduce chain), que resume los resúmenes de cada fragmento del documento, sigue un proceso similar al de la cadena de mapeo, pero requiere un poco más de configuración.

In [10]:
# Creamos la plantilla de reducción

plantilla_resumir_resúmenes = """
Escribe un resumen conciso del siguiente texto, que combina varios resúmenes, e incluye los detalles principales.
Texto: {resumenes}
"""
 
resumir_resumenes_prompt = PromptTemplate.from_template(plantilla_resumir_resúmenes)


In [11]:
# Ahora, vamos a configurar la cadena de reducción (reduce chain), que sintetiza en uno los resúmenes de cada fragmento del documento

cadena_reducir_resumenes = (
    RunnableLambda(lambda x: 
        {
            'resumenes': '\n'.join([i['resumen'] for i in x]), 
        })
    | resumir_resumenes_prompt 
    | llm 
    | StrOutputParser()
)

In [12]:
# Finalmente, vamos a encadenar todos los componentes para crear el pipeline completo

cadena_map_reduce = (
   text_chunks_chain
   | cadena_mapa_resumir.map()
   | cadena_reducir_resumenes
) 

# La función map() en cadena_mapa_resumir es esencial para habilitar el procesamiento paralelo de los fragmentos.

In [14]:
# Ahora vamos a correr la cadena

resumen_final = cadena_map_reduce.invoke(book)

In [15]:
print(resumen_final)

En "Cien años de soledad" de Gabriel García Márquez, la historia gira en torno a la familia Buendía en la aldea de Macondo, fundada por José Arcadio Buendía y su esposa Úrsula. A lo largo del relato, José Arcadio se obsesiona con el conocimiento y la ciencia, impulsado por la llegada de gitanos como Melquíades, quien le introduce inventos fascinantes. Su búsqueda de la alquimia y la transformación de metales en oro resulta en fracasos y descuidos familiares, afectando su relación con Úrsula y sus hijos, especialmente Aureliano, quien muestra un talento natural.

La narrativa también explora la vida en Macondo, desde su fundación hasta la llegada de nuevos personajes como Rebeca, una niña huérfana con costumbres peculiares, y la epidemia de insomnio que amenaza a la familia, provocando el olvido de su identidad. A medida que José Arcadio se sumerge en su obsesión, su familia enfrenta tensiones y desafíos, reflejando la lucha entre el deseo de conocimiento y las responsabilidades familia